<a href="https://colab.research.google.com/github/sivianil/Structural-Bioinformatics/blob/main/Rotation_RMSD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import math
from numpy.linalg import linalg

In [ ]:
mod1_atoms = []
mod1_coord = []
#open xyz file for reading using open() function
xyz = open('1zta_ca_1.xyz')
n_atoms = int(xyz.readline())
#reads the file line by line
title = xyz.readline()
for line in xyz:
  atom,x,y,z = line.split()
  mod1_atoms.append(atom)
  mod1_coord.append([float(x), float(y), float(z)])
xyz.close()

In [ ]:
mod2_atoms = []
mod2_coord = []
xyz = open('1zta_ca_2.xyz')
n_atoms = int(xyz.readline())
title = xyz.readline()
for line in xyz:
  atom,x,y,z = line.split()
  mod2_atoms.append(atom)
  mod2_coord.append([float(x), float(y), float(z)])
xyz.close()

In [ ]:
dic1 = {}
for i in range(len(mod1_coord)):
  val = mod1_coord[i]
  dic1[mod1_atoms[i]] = val

In [ ]:
dic2 = {}
for i in range(len(mod2_coord)):
  val = mod2_coord[i]
  dic2[mod2_atoms[i]] = val

In [ ]:
def centre_of_mass(dic):
  #centre of mass coordinates of xyz file

  cm_x, cm_y, cm_z = [], [], []
  for key, value in dic.items():
    cm_x.append(value[0])
    cm_y.append(value[1])
    cm_z.append(value[2])
  
  cm_x = sum(cm_x)/len(mod1_coord)
  cm_y = sum(cm_y)/len(mod1_coord)
  cm_z = sum(cm_z)/len(mod1_coord)

  return cm_x,cm_y,cm_z

In [ ]:
#centroid coordinates for model 1
cm_model1 = centre_of_mass(dic1)
cm_model1

(0.17048571428571405, -0.5047142857142854, -0.40911428571428576)

In [ ]:
#centroid coordinates for model 2
cm_model2 = centre_of_mass(dic2)

In [ ]:
cm_model2

(9.16122857142857, -7.341657142857142, -10.58485714285714)

Try to bring the RMSD as lows as possible. Best alignment

In [ ]:
def rad_gyration(res, dic):
  #determine the perpendicular distances of all atoms from the axis of rotation
  tot = 0
  res = np.array(res)
  for key, value in dic.items():
    rk = np.array(value)
    tot += np.linalg.norm(rk - res)**2
  
  tot /= len(mod1_coord)
  rad = np.sqrt(tot)

  return rad


In [ ]:
#radius of gyration for model1
rad_gyration(cm_model1, dic1)

14.783192944719906

In [ ]:
#radius of gyration for model2
rad_gyration(cm_model2, dic2)

15.052060455387931

In [ ]:
#center both models to zero
mod1_coord = mod1_coord - np.array(cm_model1)
mod2_coord = mod2_coord - np.array(cm_model2)

In [ ]:
m1_cord = pd.DataFrame(mod1_coord, index = mod1_atoms, columns=['x', 'y', 'z'])
m2_cord = pd.DataFrame(mod2_coord, index = mod2_atoms, columns=['x', 'y', 'z'])

In [ ]:
m1_cord.head()

,x,y,z
C0,10.814514,20.947714,0.266114
C1,8.220514,18.307714,-0.821886
C2,6.819514,16.458714,2.241114
C3,7.379514,18.985714,5.053114
C4,4.180514,18.267714,7.137114


In [ ]:
m2_cord.head()

,x,y,z
C0,24.376771,6.641657,1.339857
C1,23.747771,3.104657,-0.044143
C2,21.317771,2.573657,-3.000143
C3,18.458771,0.241657,-4.088143
C4,19.850771,-2.009343,-1.293143


In [ ]:
MI_vec = []
with open('inertiaVectors.txt') as f:
  for line in f:
    _,_,_,_,x,y,z = line.split()
    MI_vec.append([float(x), float(y), float(z)])

In [ ]:
MI_vec

[[0.541059, -0.299595, 0.78581],
 [0.787317, -0.103191, -0.581439],
 [0.010829, 0.039588, 0.007637],
 [0.027761, -0.103066, 0.994287],
 [-0.010364, -0.974455, -0.10072],
 [0.050197, -0.000385, -0.001441]]

In [ ]:

#Moment of Inertia vectors perpendicular to each other
dot = np.dot(MI_vec[0], MI_vec[1])
math.degrees(np.arccos(dot/(linalg.norm(MI_vec[0])*linalg.norm(MI_vec[1]))))

90.00000723305419

In [ ]:
m1_v1 = MI_vec[0]
m1_v2 = MI_vec[1]
m1_v3 = MI_vec[2]
m2_v1 = MI_vec[3]
m2_v2 = MI_vec[4]
m2_v3 = MI_vec[5]

In [ ]:
mod1_mat = np.array([MI_vec[0], MI_vec[1], MI_vec[2]])

In [ ]:
mod2_mat = np.array([MI_vec[3], MI_vec[4], MI_vec[5]])

In [ ]:
#eigen values and eigen vectors of model 1
np.linalg.eig(mod1_mat)

(array([0.08648591+0.37851805j, 0.08648591-0.37851805j,
        0.27253319+0.j        ]),
 array([[-0.23803303-0.36120612j, -0.23803303+0.36120612j,
          0.51253086+0.j        ],
        [-0.89614793+0.j        , -0.89614793-0.j        ,
          0.84592734+0.j        ],
        [-0.02997543+0.09429097j, -0.02997543-0.09429097j,
          0.14737384+0.j        ]]))

In [ ]:
#eigen values and eigen vectors of model 2
np.linalg.eig(mod2_mat)

(array([ 0.23849158, -0.21157378, -0.9750528 ]),
 array([[ 0.97848514, -0.9724925 ,  0.10727641],
        [-0.02536277, -0.01745521,  0.99421596],
        [ 0.20475245,  0.23227925, -0.00513776]]))

In [ ]:
a = np.array(MI_vec[0])/np.linalg.norm(MI_vec[0])
b = np.array(MI_vec[3])/np.linalg.norm(MI_vec[3])
orth_vec = np.cross(a, b)
orth_vec

array([-0.2168932 , -0.51615325, -0.04744775])

In [ ]:
b

array([ 0.027761  , -0.103066  ,  0.99428704])

In [ ]:
orth_vec = orth_vec/np.linalg.norm(orth_vec)
np.linalg.norm(orth_vec)

0.9999999999999999

In [ ]:
def skew(V):
  #function returns the skew symmetric cross product matrix of a vector
  vec = np.array([[0, -V[2], V[1]],
                  [V[2], 0, -V[0]],
                  [-V[1], V[0], 0]])
  return vec

In [ ]:
def rotation(V1, V2):

  #normalize the given MI vectors to unit vectors
  a = np.array(V1)/np.linalg.norm(V1)
  b = np.array(V2)/np.linalg.norm(V2)
  #rotation axis perpendicular to both vector a and b determined by computing cross product of a and b
  orth_vec = np.cross(a, b)
  #determining the sin and cos angles of two vectors 
  r1 = np.dot(a,b)
  #magnitude of cross product of unit vectors a and b.
  r2 = np.linalg.norm(orth_vec)
  orth_vec = orth_vec/np.linalg.norm(orth_vec)

  #determining the angle
  #angle = math.degrees(np.arccos(np.dot(a, b)))
  #radians = angle * np.pi / 180

  #rotation matrices rotate vector by an angle theta about the x, y and z-axis.
  I = np.eye(3, dtype = 'float')
  out = np.outer(orth_vec, orth_vec)
  R = r1 * I + r2 * skew(orth_vec) + (1-r1)*out

  return R

In [ ]:
rot1 = rotation(m1_v1, m2_v1)
rot1

array([[ 0.85296486,  0.10871578, -0.51052114],
       [ 0.01382028,  0.97302242,  0.23029625],
       [ 0.52178535, -0.20349015,  0.82845145]])

In [ ]:
def new_cord(m1_cord, rot_mat):
  first_rot = []
  for i in range(len(m1_cord)):
    atm = np.array(m1_cord.iloc[i,:])
    rot_co = np.dot(rot_mat, atm)
    first_rot.append([rot_co[0], rot_co[1], rot_co[2]])

  return np.array(first_rot)


In [ ]:
fst_rot = new_cord(m1_cord, rot1)
fst_rot = pd.DataFrame(fst_rot, index = mod1_atoms, columns=['x', 'y', 'z'])

In [ ]:
rot2 = rotation(m1_v2, m2_v2)
rot2

array([[ 0.44427394,  0.75267014,  0.48590979],
       [-0.8409443 ,  0.16333515,  0.51588207],
       [ 0.30892289, -0.63781603,  0.70551921]])

In [ ]:
rot2 = rotation(m1_v2, m2_v2)
sec_rot = new_cord(fst_rot, rot2)
sec_rot = pd.DataFrame(sec_rot, index = mod1_atoms, columns=['x', 'y', 'z'])

In [ ]:
sec_rot.head()

,x,y,z
C0,21.327340,-5.368711,-8.494279
C1,17.479962,-5.086245,-8.485623
C2,16.387873,-1.653013,-7.150021
C3,19.452377,0.517438,-7.860397
C4,17.633720,3.834405,-8.744927


**Since rotation matrices 1 and 3 generated from the MI vectors with eigen value of highest mangitude, I used it to rotate the CA coordiantes of both model 1 and 2. Before the third rotation, models gets superimposed**

In [ ]:
rot3 = rotation(m1_v3, m2_v3)
fin_rot = new_cord(fst_rot, rot3)
fin_rot = pd.DataFrame(fin_rot, index = mod1_atoms, columns=['x', 'y', 'z'])

In [ ]:
fin_rot.head()

,x,y,z
C0,22.626440,-5.351667,-3.903221
C1,19.084241,-4.080132,-4.750688
C2,17.757715,-1.824264,-1.937334
C3,20.977011,-0.564882,-0.307375
C4,19.813839,3.030147,0.595385


In [ ]:
#function computes the root mean square deviation of two data frames
def rmsd(df1, df2):
  x_comp, y_comp, z_comp = 0,0,0
  X0 = df1.iloc[:, 0]
  Y0 = df1.iloc[:, 1]
  Z0 = df1.iloc[:, 2]
  X1 = df2.iloc[:, 0]
  Y1 = df2.iloc[:, 1]
  Z1 = df2.iloc[:, 2]
  for i in range(len(df1)):
    x_comp += (X0[i] - X1[i])**2
    y_comp += (Y0[i] - Y1[i])**2
    z_comp += (Z0[i] - Z1[i])**2
  tot = x_comp + y_comp + z_comp
  N = len(df1)

  return np.sqrt(tot / N)

In [ ]:
rmsd(fin_rot, m2_cord)

5.267168150830916

**The final root mean square deviation between coordiantes of model1 and model 2 is ~5.153**